In [ ]:
import os
import sys
from pathlib import Path
from tqdm import tqdm
from easydict import EasyDict
import pdb
from dataset.LFWDataset import LFWDataset
from dataset.TripletFaceDataset import TripletFaceDataset
from dataset.TripDataset import TripDataset
from datetime import datetime
import torch
from torch import optim
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from utils import de_preprocess
from model.MTCNN import MTCNN
from model.Cosface import Cosface

In [ ]:
conf = EasyDict()
conf.use_cuda = torch.cuda.is_available()
conf.pin_mem = conf.use_cuda
conf.input_size = (128,92)
conf.batch_size = 128
conf.num_classes = 51332
conf.train_path = Path('/home/f/msceleb_train_cropped')
conf.val_path = Path('/home/f/nvme/msceleb_val_cropped')
conf.test_path = Path('/home/f/nvme/msceleb_test_cropped')
conf.lfw_path = '/home/f/learning/Dataset/LFW/lfw-deepfunneled'
conf.lfw_pairs_path = '/home/f/learning/Face/facenet_pytorch-master/lfw_pairs.txt'
conf.model_path = Path('/home/f/learning/Face/model_dict')
conf.log_path = Path('/home/f/learning/Face/log')
conf.lfw_path = '/home/f/nvme/lfw-deepfunneled'
conf.embedding_size = 512
conf.board_loss_every = 500
conf.eva_every = 5000
conf.save_every = 30000
conf.board_loss_every = 5
conf.eva_every = 5
conf.save_every = 5
conf.transforms_aug = trans.Compose([
    trans.Resize(conf.input_size),
    trans.RandomGrayscale(),
    trans.ColorJitter(0.2,0.2,hue=0.1),
    trans.RandomHorizontalFlip(),
    trans.ToTensor(),
    trans.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
conf.transforms = trans.Compose([
    trans.Resize(conf.input_size),
    trans.ToTensor(),
    trans.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
mtcnn = MTCNN()
conf.lfw_transforms = trans.Compose([
    trans.Lambda(lambda x : mtcnn.detect_faces(x)),
    trans.Resize(conf.input_size),
    trans.ToTensor(),
    trans.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
train_dataset = ImageFolder(conf.train_path,conf.transforms_aug)
conf.num_classes = len(train_dataset.classes)
train_loader = DataLoader(train_dataset,conf.batch_size,
                          shuffle=True,pin_memory=conf.pin_mem,drop_last=True,num_workers=1)

In [ ]:
val_trip_dataset = TripDataset(conf.val_path,conf.transforms)
val_trip_loader = DataLoader(val_trip_dataset,conf.batch_size//2,
                          shuffle=True,pin_memory=conf.pin_mem,drop_last=True,num_workers=1)

In [ ]:
test_trip_dataset = TripDataset(conf.test_path,conf.transforms)
test_trip_loader = DataLoader(test_trip_dataset,conf.batch_size//2,
                          shuffle=True,pin_memory=conf.pin_mem,drop_last=True,num_workers=1)

In [ ]:
val_loss_dataset = ImageFolder(conf.val_path,conf.transforms)
val_loss_loader = DataLoader(val_loss_dataset,conf.batch_size//2,
                             shuffle=True,pin_memory=conf.pin_mem,drop_last=True,num_workers=1)

In [ ]:
test_loss_dataset = ImageFolder(conf.test_path,conf.transforms)
test_loss_loader = DataLoader(test_loss_dataset,conf.batch_size//2,
                              shuffle=True,pin_memory=conf.pin_mem,drop_last=True,num_workers=1)

In [ ]:
lfw_dataset = TripletFaceDataset(dir=conf.lfw_path,n_triplets=1000000,transform=conf.lfw_transforms)
lfw_loader = DataLoader(lfw_dataset,
                        batch_size=128, 
                        shuffle=True,
                        num_workers=1,
                        pin_memory=conf.pin_mem)

In [ ]:
torch.save(face_model.state_dict(),conf.model_path/'cosface_best_cpu.pth')

In [ ]:
optimizer_adam = optim.Adam(face_model.parameters(),lr=0.0001)

In [ ]:
writer = SummaryWriter(conf.log_path/str(datetime.now())[:-10])

In [ ]:
# face_model.load_state(conf.model_path/'model_step591173_2018-04-16 22:51_adam_13_epochs.pth')

In [ ]:
face_model = Cosface(conf)

In [ ]:
face_model.fit(train_loader,val_trip_loader,val_loss_loader,lfw_loader,optimizer_adam,writer,epochs=15,log='adam')

In [ ]:
face_model.load_state(conf.model_path/'model_step210000_2018-04-19 06:52_adam_best.pth')

In [ ]:
face_model.cpu()

In [ ]:
face_model.save_state(conf.model_path/'face_model_best_cpu.pth')